#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
*   https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
*    https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)

#EJEMPLO 1: Prediciendo un texto (cuento los tres cerditos).

In [ ]:
!kill -9 -1 #reiniciando la maquina virtual

##P0. importando librerias

###Librerias genericas

In [ ]:
!pip list

In [ ]:
import requests
import io
import os

import numpy as np
import sys

#librerías para graficar
import matplotlib.pyplot as plt
                 
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
#guardar las imagenes y tablas en el cuaderno de jupyter
%matplotlib inline 

###librerias para DL (Deep Learning)

In [ ]:
import tensorflow as tf

In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
GPU is available


Verificando los recursos de maquina para entrenar:

---
**CUDA (GPU-NVIDIA):**
CUDA es una plataforma de computación paralela y un modelo de programación que hace que el uso de una GPU para la computación de propósito general sea simple y elegante

In [ ]:
print("-------------------------------RAM------------------------------")
!cat /proc/meminfo  #cuanta memoria tenemos?
print("-----------------------------PROCESADOR-------------------------")
!cat /proc/cpuinfo  # que procesador tenemos?
print("-------------------------------CUDA-----------------------------")
!nvcc -V   # version de CUDA

-------------------------------RAM------------------------------
MemTotal:       13302924 kB
MemFree:         5359492 kB
MemAvailable:   11806300 kB
Buffers:          158832 kB
Cached:          6054876 kB
SwapCached:            0 kB
Active:          1648628 kB
Inactive:        5785084 kB
Active(anon):     946916 kB
Inactive(anon):     4576 kB
Active(file):     701712 kB
Inactive(file):  5780508 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               948 kB
Writeback:             0 kB
AnonPages:       1219896 kB
Mapped:           716384 kB
Shmem:              5340 kB
KReclaimable:     290536 kB
Slab:             347908 kB
SReclaimable:     290536 kB
SUnreclaim:        57372 kB
KernelStack:        5968 kB
PageTables:        11388 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    4670444 kB
VmallocTotal:   34359738367 kB
VmallocU

###Activando la GPU

---
Logra aumentar la velocidad de entrenamiento en un 600% en PLN (RNN) y un 1000% en visión por computadores (CNN)


In [ ]:
tf.device('/GPU:0')# OJO: esto es para que tensorflow utilice la GPU (aumenta la velocidad de entrenamiento en un 600%) 


##P1. Descarga y preprocesado de los datos

In [ ]:
fileUrl     ='https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/Datasets/Panel_Txt_Files/Tres_cerditos.txt'
fileContent = tf.keras.utils.get_file('Tres_cerditos.txt',fileUrl)
texto       = open(fileContent, 'rb').read().decode(encoding='utf-8')
raw_text    = texto.lower()
print(raw_text)

16384/3231 [========================================================================================================================================================] - 0s 0us/step
había una vez tres cerditos que eran hermanos y se fueron por el mundo a conseguir fortuna. el más grande les dijo a sus hermanos que sería bueno que se pusieran a construir sus propias casas para estar protegidos. a los otros dos les pareció una buena idea, y se pusieron manos a la obra, cada uno construyó su casita.  - la mía será de paja - dijo el más pequeño-, la paja es blanda y se puede sujetar con facilidad. terminaré muy pronto y podré ir a jugar. el hermano mediano decidió que su casa sería de madera:  - puedo encontrar un montón de madera por los alrededores - explicó 
a sus hermanos, - construiré mi casa en un santiamén con todos estos troncos y me iré también a jugar.  cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el pro

##P2. pasar el texto a números

---
Sin importan el origen de la informacción (video, sonido, sensor, texto)...siempre debemos convertirlos en datos númericos.


In [ ]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int)

['\n', '\r', ' ', '!', ',', '-', '.', ':', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', 'á', 'é', 'í', 'ñ', 'ó', 'ú']
{'\n': 0, '\r': 1, ' ': 2, '!': 3, ',': 4, '-': 5, '.': 6, ':': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'f': 13, 'g': 14, 'h': 15, 'i': 16, 'j': 17, 'l': 18, 'm': 19, 'n': 20, 'o': 21, 'p': 22, 'q': 23, 'r': 24, 's': 25, 't': 26, 'u': 27, 'v': 28, 'x': 29, 'y': 30, 'z': 31, '¡': 32, 'á': 33, 'é': 34, 'í': 35, 'ñ': 36, 'ó': 37, 'ú': 38}


In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

En total hay 3141 caracteres, y el diccionario tiene un tamaño de 39 caracteres.


###P2.1 Dividimos el texto en secuencias:
---
Dividimos el texto en estas secuencias (adrede), convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente



In [ ]:
# preparar el conjunto de datos de los pares de entrada a salida codificados como enteros
seq_length = 50   #largo de las secciones de texto que usaremos para entrenar
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	print("Seq(",i,")=",raw_text[i:i + seq_length],"---->",raw_text[i + seq_length])
	seq_in 	= raw_text[i:i + seq_length]         					#secuencia de entrada
	seq_out = raw_text[i + seq_length]										#siguiente letra despues de la secuencia (la que el va a aprender)
	dataX.append([char_to_int[char] for char in seq_in])  # convertimos cada secuencia en numeros
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron ",format(n_patterns) ," secuencias texto de un tamaño de ",seq_length," caracteres" )
print("Como se ven los datos de X convertidos a números\n")
print(dataX)
print("\nComo se ven los datos de Y convertidos a números\n")
print(dataY)

##P3. preparar nuestros datos de entrenamiento

---


1.   Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM.
2.   Luego debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que utiliza la función de activación sigmoidea de forma predeterminada.
3.   por ultimo necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 54 caracteres diferentes en el vocabulario (una representación más fácil)







In [ ]:
#transformar la lista X de secuencias de entrada en la forma [muestras (3091), pasos de tiempo, características]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar (cambiar la escala de los números enteros al rango de 0 a 1 )
X = X / float(n_vocab)
# convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot.
y = tf.keras.utils.to_categorical(dataY)

##P4.Construcción del modelo RNN

---
definimos nuestro modelo LSTM: 
Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red usa deserción con una probabilidad de 20. La capa de salida es una capa densa que usa la función de activación softmax para generar una predicción de probabilidad para cada uno de los 54 caracteres entre 0 y 1.


In [ ]:
# define the LSTM model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))            #Softmax convierte un vector de valores en una distribución de probabilidad para cada uno de los 39
                                                                              #Softmax se utiliza a menudo como la activación para la última capa de una red de clasificación
#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P4.1 Creando chekpoints

---

La red es lenta de entrenar (alrededor de 300 segundos por época) teniendo activa la GPU, ASí que crearemos CHECKPOINTS (puntos de control) para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección

In [ ]:
# definimos  una carpeta para guardar los checkpoint
filepath="checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

###P4.2 entrenando

In [ ]:
history = model.fit(X, y, epochs=1000, batch_size=128, verbose=1,callbacks=callbacks_list)

##P5.Generando texto con una red LSTM


---
Vamos a cargar el ultimo CHECKPOINT de entrenamiento y con el haremos MAGIA!!!


In [ ]:
filename = "/content/checkpoints/weights-improvement-940-0.0035.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P5.1 mapeo inverso (números a letras)
creamos un mapeo inverso que podamos usar para convertir los números enteros nuevamente en caracteres para que podamos entender las predicciones

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

###P5.2 hacer predicciones
La forma más sencilla de utilizar el modelo Keras LSTM para hacer predicciones es comenzar primero con una secuencia semilla como entrada, generar el siguiente carácter y luego actualizar la secuencia semilla para agregar el carácter generado al final y recortar el primer carácter. Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud).

In [ ]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de 10000 caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"v e z   t r e s   c e r d i t o s   q u e   e r a n   h e r m a n o s   y   s e   f u e r o n   p o"
r el mundo a conseguir fortuna. el más grande les dijo a sus hermanos que sería bueno que se pusieran a construir sus propias casas para estar protegidos. a los otros dos les pareció una buena idea, y se pusieron manos a la obra, cada uno construyó su casita.  - la mía será de paja - dijo el más pequeño-, la paja es blanda y se puede sujetar con facilidad. terminaré muy pronto y podré ir a jugar. el hermano mediano decidió que su casa sería de madera:  - puedo encontrar un montón de madera por los alrededores - explicó 
a sus hermanos, - construiré mi casa en un santiamén con todos estos troncos y me iré también a jugar.  cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el problema:  -¡quién teme al lobo feroz, al lobo, al lobo!  - ¡quién teme al lobo feroz, al lobo feroz! - cantaban desde dentro los ce

##P6.Mejorando la red (una LSTM más grande)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(256))                                                                #agregaremos una segunda capa. 
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

###P6.1 mejoramos el entrenamiento

---
aumentamos las epoch y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender.


In [ ]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=32, callbacks=callbacks_list)

###P6.2 haciendo predicciones

In [ ]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
 
 a   s u s   h e r m a n o s ,   -   c o n s t r u i r é   m i   c a s a   e n   u n   s a n t"
iamén con todos estos troncos y me iré también a jugar.  cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el problema:  -¡quién teme al lobo feroz, al lobo, al lobo!  - ¡quién teme al lobo feroz, al lobo feroz! - cantaban desde dentro los cerditos.  de nuevo el lobo, más enfurecido que antes al sentirse engañado, se colocó delante de la puerta y comenzó a soplar y soplar gruñendo: - ¡cerditos, abridme la puerta! - no, no, no, no te vamos a abrir.    pues si no me abrís... ¡soplaré y soplaré y la casita derribaré! y sopló con todas sus fuerzas, sopló y sopló,y la casita de paja se vino abajo. el cerdito pequeño corrió lo más rápido que pudo y entró en la casa de madera del hermano mediano. - ¡quién teme al lobo feroz, al lobo, al lobo!  - ¡quién teme al lobo feroz, al lobo feroz! - cantaban desde dentro los 

##P7. exportar modelo RNN

In [ ]:
!pip install h5py

In [ ]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modeloRNN_cuentosPesos.hdf5")
model.save('modelRNN_cuentos_v_h5.h5')
print("modelo salvado en disco")

modelo salvado en disco


###P7.1 cargando un modelo

In [ ]:
# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('/content/modelRNN_cuentos_v_h5.h5')

In [ ]:
chars = sorted(list(set("comiendo una manzana")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

En total hay 55827 caracteres, y el diccionario tiene un tamaño de 11 caracteres.
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 18, 16, 31, 20, 33, 36, 18, 24, 35, 16, 2, 33, 30, 25, 16, 1, 0, 23, 16, 17, 50, 16, 2, 36, 29, 16, 2, 37, 20, 41, 2, 36, 29, 16, 2, 19, 36, 27, 18, 20, 2, 29, 24, 51, 16, 2, 32, 36, 20, 2, 32, 36, 20, 33, 50, 16, 2, 28, 36, 18, 23, 30, 2, 16, 2, 34, 36, 2, 28, 16, 19, 33, 20, 2, 40, 2, 16, 2, 34, 36, 2, 16, 17, 36]


In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = new_model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"n t e m p l a r   l a s   f l o r e s ,   q u e   c o m e n z a b a n   a   l l e n a r l o   t o d o .   a l l í   v i o   e n   e l   e s t a n q u e   d o s   d e   a q u e l l o s   p á j a r o s"
 sue a la belle y con la boca abierta, lus amlgos de la casa de los piratas, el pey y el gato com su camino de la casa de su harfio, el príncipe juan, pirocho y la reñora darling se quedó hacia ella.
y se precó ticme que el cartillo y se puedó hacia la casa de su harta que se encontraban en el estuvendo conmigo y sodos que el carciici y se puedó hacia ella.
el patito se sintió delta en palacio, pero el hombre de jengibre corrió más rápido. en la cocina de la casa de los piratas, el peys de noche y la reñora darling se quedó hacia la casa de los piratas,
una teca a tu padre la boca abierta dl la casa y algo suppirar y al canbio, estaba muy bueno y poder vup hijas de podenas del príncipe pue al verla. se acuirtió cnn un cama y deseruer a la princesa y lls piratas a la princesa y l



---



---



#Ejemplo 2: generar texto de cuentos, usando Keras

##Instalando librerias CUDA

In [12]:
#!pip list
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


##P0. importar librerias

In [ ]:
!pip install tensorflow-gpu

In [104]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

###P0.1 Uso de GPU para entrenar en tensorflow

---
https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa


In [5]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


###P0.2 probando rendimiento de CPU vs GPU

In [7]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)   

<tf.Tensor: shape=(), dtype=float32, numpy=-624.27527>

In [9]:
cpu()  #ejecutamos entrenamiento con CPU
gpu()  #ejecutamos entrenamiento con GPU
# Run the op several times.
print('TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes '
      '(batch x height x width x channel). suma de 10 epochs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes (batch x height x width x channel). suma de 10 epochs.
CPU (s):
3.7520549730000994
GPU (s):
0.04652361299997665
GPU speedup over CPU: 80x


###P0.3 Dejando activo la GPU

In [14]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU 

##P0. Descarga y preprocesado de los datos

In [62]:
fileDL= tf.keras.utils.get_file('LasMinasDelReySalomon.txt','https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/Datasets/Panel_Txt_Files/LasMinasDelReySalomon.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')


###P0.1 limpiar/normalizar el texto

---
*   pasar todo a minusculas
*   convertir tildes a vocales sin tilde
*   eliminar caracteres especiales (*#!()%=
*   eliminar saltos de linea y tab (identación)

In [63]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

henry r haggard las minas del rey salomon introduccion ahora que este libro esta impreso y a punto de salir al mundo ejerce sobre mi un enorme peso la conciencia de sus defectos tanto de estilo como de contenido en lo referente a este ultimo solo puedo decir que no pretende ser una relacion exhaustiva de todo lo que vimos e hicimos hay muchas cosas concernientes a nuestro viaje a kukuanalandia en las que me hubiese gustado explayarme y a las que de hecho apenas aludo entre ellas se encuentran las curiosas leyendas que recogi sobre las armaduras que nos salvaron de la muerte en la gran batalla de loo y tambien sobre los silenciosos o colosos de la entrada de la cueva de estalactitas por otra parte si me hubiera dejado llevar por mis inclinaciones me habria gustado ahondar en las diferencias algunas de las cuales me resultan muy sugestivas entre los dialectos zulu y kukuana asimismo tambien se hubieran podido dedicar unas cuantas paginas de provecho al estudio de la flora y la fauna indi

##P1. entendiendo el texto

In [66]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:474359 caracteres
el texto esta compuesto de estos :37 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [126]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones

#for char,_ in zip(char2idx, range(len(vocab))):
    #print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'henry r haggard las minas del rey salomon introduccion ahora que este libro esta impreso y a punto d'
array([18, 15, 24, 28, 35,  0, 28,  0, 18, 11, 17, 17, 11, 28, 14,  0, 22,
       11, 29,  0, 23, 19, 24, 11, 29,  0, 14, 15, 22,  0, 28, 15, 35,  0,
       29, 11, 22, 25, 23, 25, 24,  0, 19, 24, 30, 28, 25, 14, 31, 13, 13,
       19, 25, 24,  0, 11, 18, 25, 28, 11,  0, 27, 31, 15,  0, 15, 29, 30,
       15,  0, 22, 19, 12, 28, 25,  0, 15, 29, 30, 11,  0, 19, 23, 26, 28,
       15, 29, 25,  0, 35,  0, 11,  0, 26, 31, 24, 30, 25,  0, 14])


###P2.1 exportar vocablos y matriz de numerica

---

exporta en un CSV para usarlo cuando se cargue el modelo en otra instancia

In [117]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

##P3. preparar los datos para ser usados en la RNN

In [118]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'henry r haggard las minas del rey salomon introduccion ahora que este libro esta impreso y a punto de salir al mundo ejerce sobre mi un enorme peso la '
'conciencia de sus defectos tanto de estilo como de contenido en lo referente a este ultimo solo puedo decir que no pretende ser una relacion exhaustiva'
' de todo lo que vimos e hicimos hay muchas cosas concernientes a nuestro viaje a kukuanalandia en las que me hubiese gustado explayarme y a las que de '
'hecho apenas aludo entre ellas se encuentran las curiosas leyendas que recogi sobre las armaduras que nos salvaron de la muerte en la gran batalla de l'
'oo y tambien sobre los silenciosos o colosos de la entrada de la cueva de estalactitas por otra parte si me hubiera dejado llevar por mis inclinaciones'
' me habria gustado ahondar en las diferencias algunas de las cuales me resultan muy sugestivas entre los dialectos zulu y kukuana asimismo tambien se h'
'ubieran podido dedicar unas cuantas paginas de provecho al estudio de la fl

###P3.1 separar los datos en agrupamientos (batches)

In [119]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'henry r haggard las minas del rey salomon introduccion ahora que este libro esta impreso y a punto de salir al mundo ejerce sobre mi un enorme peso la'
Target data:  'enry r haggard las minas del rey salomon introduccion ahora que este libro esta impreso y a punto de salir al mundo ejerce sobre mi un enorme peso la '


In [120]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [121]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           9472      
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 37)            37925     
                                                                 
Total params: 5,294,373
Trainable params: 5,294,373
Non-trainable params: 0
_________________________________________________________________


###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [122]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [123]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


###P4.2 entrenando

####P4.2a entrenando para usar chekpoints


---

*   https://keras.io/api/callbacks/model_checkpoint/
*   https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

In [ ]:
EPOCHS=100
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [ ]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints_test1/cp_0015.ckpt


In [ ]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=50, 
                    verbose=1,
                    callbacks=[cp_callback])

####P4.2b entrenando con tensorboard (opcional)

#####Activando TENSORBOARD 

---
(DASHBOARD para ver el proceso de entrenamiento)

In [ ]:
# You can change the directory name
LOG_DIR = 'tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-11-17 22:51:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  2.61MB/s    in 12s     

2021-11-17 22:51:50 (1.14 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
http://47d1-104-199-163-197.ngrok.io


In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

#####Fit

In [ ]:

#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)
history_TB=model2.fit(dataset, 
                    epochs=200, 
                    verbose=1,
                    callbacks=[tbCallBack])

52/52 [==============================] - 15s 283ms/step - loss: 0.1148 - accuracy: 0.9777
Epoch 195/200
52/52 [==============================] - 15s 282ms/step - loss: 0.1139 - accuracy: 0.9777
Epoch 196/200
52/52 [==============================] - 15s 283ms/step - loss: 0.1129 - accuracy: 0.9779
Epoch 197/200
52/52 [==============================] - 15s 283ms/step - loss: 0.1115 - accuracy: 0.9780
Epoch 198/200
52/52 [==============================] - 16s 283ms/step - loss: 0.1112 - accuracy: 0.9780
Epoch 199/200
52/52 [==============================] - 15s 282ms/step - loss: 0.1123 - accuracy: 0.9779
Epoch 200/200
52/52 [==============================] - 15s 282ms/step - loss: 0.1101 - accuracy: 0.9782


##P5. Generando texto nuevo usando la RNN

In [ ]:
#creamos un modelo tomando como base el ultimo checkpoint
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [ ]:
print(generate_text(model, start_string=u"la colonia europea"))

la colonia europea, que siguió a los heroísmos de los primeros exploradores, fue 
acompañado, en otro plano, por la aventura. la aventura personal. en los primeros libros de 
melancolía, y por extraño que parezca, no nos sentíamos 
demasiado mal tras la terrible experiencia del valor y atractivo aventurero); y por fin, inevitablemente, 
una lucha con la muerte, que es a la vez concreta y una alegoría ún que el pueblo kukuana preserva a sus reyes 
muertos desde tiempo inmemorial. los petrifica. no llegué a descubr


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [ ]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


##P7.Cargando un modelo serializado

###P7.1 descargamos el modelo usando wget

In [ ]:
!wget https://github.com/michelBolivar/Deep_learning/blob/main/Corte2/cuento/cuento_1500/modelRNN_cuentos2_2.h5?raw=true \
      -O modelRNN_cuentos2_2.h5

####P7.1a descargamos el modelo usando PYRIND & URLLIB (OPCIONAL)

In [ ]:
!pip install pyprind

In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/luisFernandoCastellanosG/Machine_learning/blob/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/RNN_LasMinasDelReySalomon_model.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_LasMinasDelReySalomon_model2.h5', 
                           reporthook)

100% | 61 MB | 4.15 MB/s | 14 segundos transcurrido

('RNN_LasMinasDelReySalomon_model2.h5',
 <http.client.HTTPMessage at 0x7fa210031590>)

###P7.2 instanciamos el modelo descargado

In [ ]:
new_model = tf.keras.models.load_model('/content/modelRNN_cuentos2_2.h5')

In [125]:
df2 = pd.read_csv("https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/data_vocab.csv")
df2.head()

,num,vocab
0,0,
1,1,0
2,2,1
3,3,2
4,4,3


In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval  = [char2idx[s] for s in start_string]
  input_eval  = tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


In [ ]:
print(generate_text(new_model, start_string=u"los fantasmas de "))

los fantasmas de 9a 7dgdc9a 9g5 jc ?db6g9 5a9;g9 n _dk\5a) 5hw 7dbd hj 9hedh5 9g5 a5 ig5iy 89 7d;9ga5 9c hjh 6g5odh- e9gd 9c 9a b\hbd \chi5ci9 589a;jc85 h9 5bedaa5 h9 gdbew5 89 :dgb5 \cd7j5 7dcig5 a5h e\98g5h 89a e5h58\od b\9cig5h 9a zai\bd n e5ivi\7d g9o5;58d) 9a d69hd 7d7\c9gd) 89h5e5g97w5 e9h585b9ci9 89 a5 k\hi5 9c 9a hdb6gwd 9hi5cfj9) a5h :a5ji5h n i5b6dg9h 9cbj897\9gdc) n adh 79;58dg9h g5ndh 89 a5h gj\c5h h9 9h:jb5gdc 5a \chi5ci9) 89_5c8d a5 5a895 89 a5 b5a8\7\yc 89hda585 n hda\i5g\5 65_d adh i9cj9h g5ndh 8
